In [4]:
import pandas as pd
import requests
import xml.etree.ElementTree as ET
from shapely.geos import lgeos
from google.cloud import bigquery
from bs4 import BeautifulSoup
import time
import datetime as dt
import re

In [5]:
# Set up BigQuery client and credentials
client = bigquery.Client.from_service_account_json('YOUR_BIGQUERY_CREDENTIALS')

route_map = {'y': 'Yellow Line',
 'org': 'Orange Line',
 'pink': 'Pink Line',
 'brn': 'Brown Line',
 'g': 'Green Line',
 'red': 'Red Line',
 'blue': 'Blue Line',
 'p': 'Purple Line'}


# Define function to get data from CTA API and create dataframe
def get_cta_data():
    url = 'http://lapi.transitchicago.com/api/1.0/ttpositions.aspx'
    params = {'key': 'YOUR_CTA_MAP_ID', 'rt': ['red,blue,brn,g,org,p,pink,y'], 'outputType': 'xml'}
    response = requests.get(url, params=params)
    soup = BeautifulSoup(response.content, 'xml')
    train_data = []
    for route in soup.find_all('route'):
        for train in route.find_all('train'):
            train_dict = {'route': str(route_map[route['name']]),
                          'run_number': int(train.rn.text),
                          'destination_station_id': int(train.destSt.text),
                          'destination_station_name': str(train.destNm.text),
                          'train_direction': int(train.trDr.text),
                          'next_station_id': int(train.nextStaId.text),
                          'next_stop_id': int(train.nextStpId.text),
                          'next_station_name': str(train.nextStaNm.text),
                          'timestamp': dt.datetime.strptime(train.prdt.text, '%Y%m%d %H:%M:%S'),
                          'arrival_time': dt.datetime.strptime(train.arrT.text, '%Y%m%d %H:%M:%S'),
                          'is_approaching': int(train.isApp.text),
                          'is_delayed': int(train.isDly.text),
                          'latitude': float(train.lat.text),
                          'longitude': float(train.lon.text),
                          'heading': float(train.heading.text)}
            train_data.append(train_dict)
    df = pd.DataFrame(train_data)
    return df

# Define function to upload dataframe to BigQuery
def upload_to_bigquery(df):
    table_id = "YOUR_BIGQUERY_PROJECT_ID"
    job_config = bigquery.LoadJobConfig(
        schema=[
            bigquery.SchemaField('route', 'STRING'),
            bigquery.SchemaField('run_number', 'INTEGER'),
            bigquery.SchemaField('destination_station_id', 'INTEGER'),
            bigquery.SchemaField('destination_station_name', 'STRING'),
            bigquery.SchemaField('train_direction', 'INTEGER'),
            bigquery.SchemaField('next_station_id', 'INTEGER'),
            bigquery.SchemaField('next_stop_id', 'INTEGER'),
            bigquery.SchemaField('next_station_name', 'STRING'),
            bigquery.SchemaField('timestamp', 'DATETIME'),
            bigquery.SchemaField('arrival_time', 'DATETIME'),
            bigquery.SchemaField('is_approaching', 'INTEGER'),
            bigquery.SchemaField('is_delayed', 'INTEGER'),
            bigquery.SchemaField('latitude', 'FLOAT'),
            bigquery.SchemaField('longitude', 'FLOAT'),
            bigquery.SchemaField('heading', 'FLOAT')
        ],
        write_disposition='WRITE_TRUNCATE'
    )
    job = client.load_table_from_dataframe(df, table_id, job_config=job_config)
    job.result()  # Wait for job to complete
    print('Uploaded {} rows to BigQuery table {}'.format(len(df), table_id))

# Loop to get data and upload to BigQuery every 30 seconds
while True:
    df = get_cta_data()
    upload_to_bigquery(df)
    time.sleep(30)

Uploaded 77 rows to BigQuery table future-sonar-331521.cta_tracker.train_data
Uploaded 78 rows to BigQuery table future-sonar-331521.cta_tracker.train_data
Uploaded 76 rows to BigQuery table future-sonar-331521.cta_tracker.train_data



KeyboardInterrupt

